In [ ]:
## Necessary imports
import pandas as pd
import numpy as np
import xlsxwriter

# -------------------------------
# Variables generales
# -------------------------------
tipo = "TB"
model = "with HDD"
tiempo = "anual"
version = "V2"

# Carpeta base (pasar desde el script principal)
# Ejemplo: ruta_modificado = 'C:/Users/Public/Escritorio/QGIS_Araba/2.Archivos Modificados'

# -------------------------------
# 1️⃣ Cargar predicciones TB generadas
# -------------------------------
csv_tb = os.path.join(ruta_modificado, 'Predicciones_Edificios_TB.csv')
df = pd.read_csv(csv_tb)
base_temp = df['predictions']  # columna generada por el script

In [ ]:
# -------------------------------
# 2️⃣ Cargar datos climáticos
# -------------------------------
city = "Bilbao"
archivo_climatico = os.path.join(ruta_modificado, f"ArchivoClimatico{city.upper()}.xls")
df_temp = pd.read_excel(archivo_climatico)
df_temp = df_temp.set_index('Date/Time')
ts_temp = df_temp['OutTemp']

# -------------------------------
# 3️⃣ Calcular HDD para cada edificio
# -------------------------------
hdd_tot = []
for tb in base_temp:
    tb = float(tb)
    df_temp['HDD'] = tb - ts_temp
    df_temp['Logico'] = np.where(df_temp['HDD'] < 0, 0, 1)
    df_temp['HDD2'] = df_temp['HDD'] * df_temp['Logico']
    hdd_tot.append(df_temp['HDD2'].sum() / 24)

# -------------------------------
# 4️⃣ Guardar TB y HDD
# -------------------------------
output_file_hdd = os.path.join(ruta_modificado, f'TByHDD_modelo_{tipo}_{model}_{tiempo}_{version}.xlsx')
workbook = xlsxwriter.Workbook(output_file_hdd)
worksheet1 = workbook.add_worksheet("Resultados")
header = ['TB', 'HDD']

# Header
for col, h in enumerate(header):
    worksheet1.write(0, col, h)

# TB y HDD
for row, tb in enumerate(base_temp, start=1):
    worksheet1.write(row, 0, float(tb))
    worksheet1.write(row, 1, hdd_tot[row-1])

workbook.close()
print(f'HDD value export is done: {output_file_hdd}')

In [ ]:
# -------------------------------
# 5️⃣ Calcular demanda acumulada
# -------------------------------
ua_file = os.path.join(ruta_modificado, f'gorbeialdea_domestico_{version}.xlsx')
df_UA = pd.read_excel(ua_file)
sigma = df_UA['UA +  ρVcp (KW/K)']

HDD = np.array(hdd_tot)
PDemand = sigma.values * HDD
PredDemand = PDemand * 24  # kWh

# -------------------------------
# 6️⃣ Exportar resultados acumulados
# -------------------------------
output_file_pred = os.path.join(ruta_modificado, f'Prediction_Gorbeialdea_{version}_TB.xlsx')
dframe = pd.DataFrame({
    'CodEdifici': df['CodEdifici'],
    'TB Model Prediction (ºC)': base_temp,
    'UA +  ρVcp (KW/K)': sigma,
    'HDD': HDD,
    'Prediction (kWh)': PredDemand
})
dframe.to_excel(output_file_pred, index=False)
print(f'Cumulative prediction result is exported: {output_file_pred}')